In [10]:
import pandas as pd
# import polars as pl

In [11]:
data_path = r"C:\Users\parth\2023\Fall 2023\Projects\docker_hack\data\data.csv"
# data_path = "../../../data/csv_dumps/data.csv"

In [12]:
df = pd.read_csv(data_path)
# df = pl.read_csv(data_path)#, encoding='latin-1')
df.head()

,first,actors,details,boxOffice,techspecs
0,"[{'title': 'Mobile Two'}, {'genres': ['Drama']...",[{'image': 'https://m.media-amazon.com/images/...,[{'title-details-releasedate': {'val': ['Septe...,[],[{'title-techspec_runtime': {'val': '1 hour 30...
1,"[{'title': 'Giving Becky a Chance'}, {'poster_...",[{'image': 'https://m.media-amazon.com/images/...,[{'title-details-releasedate': {'val': ['June ...,[],[{'title-techspec_runtime': {'val': '50 minute...
2,"[{'title': 'Promises'}, {'poster_link': 'https...",[],[{'title-details-releasedate': {'val': ['2022 ...,"[{'title-boxoffice-budget': {'val': '$8,000 (e...",[{'title-techspec_runtime': {'val': '14 minute...
3,"[{'title': 'Sin'}, {'poster_link': 'https://m....",[{'image': 'https://m.media-amazon.com/images/...,[{'title-details-releasedate': {'val': ['Octob...,[],[{'title-techspec_runtime': {'val': '50 minute...
4,"[{'title': 'The Miseducation of Bindu'}, {'pos...",[{'image': 'https://m.media-amazon.com/images/...,[{'title-details-releasedate': {'val': ['Octob...,[],[{'title-techspec_runtime': {'val': '1 hour 32...


In [13]:
#for debug
# eval(df["boxOffice"][17])
eval(df["first"][17])

[{'title': 'Listening in the Dark'},
 {'poster_link': 'https://m.media-amazon.com/images/M/MV5BY2U2MzhjNTctMWFmMS00YzE5LWIyMDMtZmEwZDMwNDU4OWZiXkEyXkFqcGdeQXVyNDA4MDE4Nw@@._V1_QL75_UY281_CR151,0,190,281_.jpg'},
 {'genres': ['Documentary', 'Short']},
 {'plot': 'Human beings are horribly blind to the effect that they have on the natural environment. Yet all around us those repercussions reverberate.'}]

In [14]:
def make_new_columns_boxOffice(curr_df,col_name, list_of_keys):
    """
    params will be(col_name,[list_of_keys]) ,
    the names in the list should be same as keys
    """
    if(col_name not in curr_df.columns):
        print("No such column exists in the df")
    for key in list_of_keys:
        curr_df[key] = ""
    for i in range(len(curr_df)):
        response_list = eval(curr_df["boxOffice"][i])
        if(response_list):
            response = response_list
            for it in response:
                for key in list_of_keys:
                    if key in it:
                        curr_df[key][i] = it[key]["val"]
                        

    return curr_df
    
    # delete the original column 

def make_new_columns_first(curr_df, col_name, list_of_keys):
    """
    params will be(col_name,[list_of_keys]) ,
    the names in the list should be same as keys
    """
    if (col_name not in curr_df.columns):
        print("No such column exists in the df")
    for key in list_of_keys:
        curr_df[key] = ""
    for i in range(len(curr_df)):
        response_list = eval(curr_df["first"][i])
        if (response_list):
            response = response_list
            for it in response:
                for key in list_of_keys:
                    if key in it:
                        curr_df[key][i] = it[key]
                        

    return curr_df
    
    # delete the original column 

# def insert_column_num_actors(curr_df):
#     """
#     Inserts a new column that states how many actors are in a particular movie.
#     """
#     curr_df['num_actors'] = pd.Series(dtype='int')
#     for i in range(len(curr_df)):
#         response_list = eval(curr_df["actors"][i])
#         if (response_list):
#             curr_df['num_actors'][i] = len(response_list)
#         #     response = response_list
#         #     for it in response:
#         #         for key in list_of_keys:
#         #             if key in it:
#         #                 curr_df[key][i] = it[key]
                        

#     return curr_df
    
    # delete the original column 
    

In [15]:
new_df = df
new_df = make_new_columns_boxOffice(new_df,"boxOffice", ["title-boxoffice-budget","title-boxoffice-cumulativeworldwidegross","title-boxoffice-grossdomestic","title-boxoffice-openingweekenddomestic"])
new_df = make_new_columns_first(new_df,'first',['title','poster_link','genres','plot','imdb-rating'])
new_df.drop(['first','boxOffice','poster_link'], axis = 1, inplace = True)

In [16]:
# pre-processing steps

# capture only the imdb rating, remove /10 part and convert to float, insert NaN where no value exists
new_df['imdb_rating'] = new_df['imdb-rating'].str.split('/').str[0]
new_df['imdb_rating'] = pd.to_numeric(new_df['imdb_rating'], errors='coerce').astype(float)

# create new column #genres, fills in 0 where no value is present
new_df['num_genres'] = new_df['genres'].apply(lambda x: len(x))

# create new column #actors, fills in 0 where no value is present
new_df['num_actors'] = new_df['actors'].apply(lambda x: len(eval(x)))

In [34]:
def money_str_to_currency(input_str):
    '''Takes an input as a str. Saves currency signs if any\n",
    saves value'''
    if(len(input_str)==0):
        return None
    #strip the str of any spaces\n",
    stripped_input_str = input_str.strip()
    split_str = stripped_input_str.split()
    money_str = split_str[0]
    currency = money_str[:1]
    # money_with_comma = money_str[1:]
    # money = money_with_comma.replace(",","")
    return currency

In [35]:
def money_str_to_int(input_str):
    '''Takes an input as a str. Saves currency signs if any\n",
    saves value'''
    if(len(input_str)==0):
        return None
    #strip the str of any spaces\n",
    stripped_input_str = input_str.strip()
    split_str = stripped_input_str.split()
    money_str = split_str[0]
    currency = money_str[:1]
    money_with_comma = money_str[1:]
    money = money_with_comma.replace(",","")
    return money

In [36]:
new_df["currency"] = None
new_df["money"] = None

In [37]:
new_df["currency"] = new_df["title-boxoffice-budget"].apply(money_str_to_currency)
new_df["money"] = new_df["title-boxoffice-budget"].apply(money_str_to_int)

In [39]:
df["techspecs"][0]

"[{'title-techspec_runtime': {'val': '1 hour 30 minutes'}}, {'title-techspec_soundmix': {'val': ['Mono']}}, {'title-techspec_aspectratio': {'val': ['1.33 : 1']}}]"

In [53]:
new_df["runtime"] = None
new_df["soundmix"] = None
new_df["aspect_ratio"] = None

In [54]:
def extract_techspecs(input_str):
    try:
        if(len(input_str)==0):
            return [None,None,None]
        input_list = eval(input_str)
        '''iterate on every object,
        find every key in every object,
        if you find it place the value 
        else place None'''
        runtime = None
        soundmix = None
        aspect_ratio = None
        for obj in input_list:
            if("title-techspec_runtime" in obj):
                runtime = obj["title-techspec_runtime"]["val"]
            if("title-techspec_soundmix" in obj):
                soundmix = obj["title-techspec_soundmix"]["val"]
            if("title-techspec_aspectratio" in obj):
                aspect_ratio = obj["title-techspec_aspectratio"]["val"]
    except Exception as e:
        print(input_str,e)
        sys.exit(1)
    return [runtime,soundmix,aspect_ratio]
        
            

In [62]:
for i in range(len(new_df)):
    input_str = new_df["techspecs"][i]
    runtime,soundmix,aspect_ratio = extract_techspecs(input_str)
    new_df["runtime"][i] = runtime
    new_df["soundmix"][i] = soundmix
    new_df["aspect_ratio"][i] = aspect_ratio
    

C:\Users\parth\AppData\Local\Temp\ipykernel_25136\734792789.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["runtime"][i] = runtime
C:\Users\parth\AppData\Local\Temp\ipykernel_25136\734792789.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["soundmix"][i] = soundmix
C:\Users\parth\AppData\Local\Temp\ipykernel_25136\734792789.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["aspect_ratio"][i] = aspect_rati

In [63]:
# new_df["runtime"] = new_df["techspecs"].apply(extract_techspecs)[0]
# new_df["soundmix"] = new_df["techspecs"].apply(extract_techspecs)[1]
# new_df["aspect_ratio"] = new_df["techspecs"].apply(extract_techspecs)[2]

In [66]:
new_df.columns

Index(['actors', 'details', 'techspecs', 'title-boxoffice-budget',
       'title-boxoffice-cumulativeworldwidegross',
       'title-boxoffice-grossdomestic',
       'title-boxoffice-openingweekenddomestic', 'title', 'genres', 'plot',
       'imdb-rating', 'imdb_rating', 'num_genres', 'num_actors', 'currency',
       'money', 'runtime', 'soundmix', 'aspect_ratio'],
      dtype='object')

In [71]:
def extract_columns_from_dataframe(df,column_list):
    """provide a dataframe and a column list,
    return the subset of the datafram if the column exists"""
    column_confirmed_list = []
    for column in column_list:
        if column in df.columns:
            column_confirmed_list.append(column)
        else:
            print("{} column not in the dataframe\n".format(column))
    df = df[column_confirmed_list]
    return df

In [72]:
new_df[["title"]]

,title
0,Mobile Two
1,Giving Becky a Chance
2,Promises
3,Sin
4,The Miseducation of Bindu
...,...
100394,Ron Hopper's Misfortune
100395,Coins for Christmas
100396,Alpha Delta Zatan
100397,Quadratic Equations


In [73]:
column_list = ['title', 'genres', 'plot',
       'imdb-rating', 'imdb_rating', 'num_genres', 'num_actors', 'currency',
       'money', 'runtime', 'soundmix', 'aspect_ratio']

In [74]:
extract_df = extract_columns_from_dataframe(new_df,column_list)

In [76]:
extract_df

,title,genres,plot,imdb-rating,imdb_rating,num_genres,num_actors,currency,money,runtime,soundmix,aspect_ratio
0,Mobile Two,[Drama],A TV reporter whose career was ruined by his d...,6.9/10,6.9,1,10,None,None,1 hour 30 minutes,[Mono],[1.33 : 1]
1,Giving Becky a Chance,[Drama],"Becky Knight's parents, simple shopkeepers, ha...",,NaN,1,5,None,None,50 minutes,[Silent],[1.33 : 1]
2,Promises,[Short],"Set in eastern Ukraine, follow the intimate an...",,NaN,1,0,$,8000,14 minutes,None,None
3,Sin,[Drama],Italian peasant girl deserts her fiancé for we...,,NaN,1,5,None,None,50 minutes,[Silent],[1.33 : 1]
4,The Miseducation of Bindu,[Comedy],When a bullied Indian teen forges her mother's...,5.6/10,5.6,1,18,None,None,1 hour 32 minutes,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
100394,Ron Hopper's Misfortune,"[Drama, Fantasy, Mystery]","Sara, a beautiful young girl, returns to an ol...",4.2/10,4.2,3,9,$,1200000,1 hour 33 minutes,None,[2.35 : 1]
100395,Coins for Christmas,"[Comedy, Drama]",Single mom Madison finds herself scraping penn...,6.4/10,6.4,2,18,None,None,1 hour 30 minutes,None,None
100396,Alpha Delta Zatan,"[Horror, Mystery, Thriller]","In a frat house where no one can be trusted, a...",3.5/10,3.5,3,16,$,50000,1 hour 12 minutes,None,[1.78 : 1]
100397,Quadratic Equations,"[Short, Comedy]",There's a substitute teacher for math class to...,,NaN,2,9,$,10000,6 minutes,None,None


In [ ]:
# df["title-boxoffice-budget"][1]

In [ ]:
df.eq('').sum() 